# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import r2_score
from numpy.linalg import inv
from scipy import linalg

In [2]:
df = pd.read_csv('insurance.csv')
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [4]:
df.duplicated().sum()

153

Мы видим 153 дубликата, но я бы не стал удалять их. На мой взгляд существует вероятность повторения данных, так как тут нет таких характерных признаков как уникальный ID или что то подобное указывающее на уникальность. Люди могут быть одного пола и возраста 

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.

Целевой признак: количество страховых выплат клиенту за последние 5 лет.

Пропусков и проблм нет, можно приступать к работе, предобработка не нужна

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

$$a' = XP ((XP)^T (XP))^{-1} (XP)^T y $$
$$a' = XP (P^T X^T (XP))^{-1} P^T X^T y $$
$$a' = XP P^{-1} (X^T X)^{-1} P^{-T} P^T X^T y $$
Где $P P^{-1}$ и $P^{-T} P^T$ это Е и Е соотвественно. Т.е. единичные матрицы
$$a' = X (X^T X)^{-1} X^T y $$
$$a' = Xw$$
$$a' = a$$

## Алгоритм преобразования

Предложим алгоритм преобразования данных для решения задачи. Обоснуем, почему качество линейной регрессии не поменяется.

**Алгоритм**

1. Сгенерировать сучайную матрицу размером 4 на 4
2. Проверить сгенерированную матрицу на обратимость
3. Умножим сгенерированную матрицу на матрицу признаков
4. Построим линейную модель для зашифрованной модели и исходной
5. Получим R2 для зашифрованной матрицы и сравним с R2 с исходными признаками

**Обоснование**

Как мы доказали выше для формулы линейной регрессии не должно быть разницы и $a = a'$, то есть умножение признаков на произвольную матрицу не должно повлиять на предсказание модели

## Проверка алгоритма

Проверим базовый вариант с данными признаками

In [5]:
target = df['Страховые выплаты']
features = df.drop(['Страховые выплаты'], axis=1)
features.shape # отметим для себя размер признаков, нам нужно сохранить его в новой матрице

(5000, 4)

In [6]:
class LinearRegression:
    def fit(self, features_train, target_train):
        X = np.concatenate((np.ones((features_train.shape[0], 1)), features_train), axis=1)
        y = target_train
        w = (np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y))
        self.w = w[1:]
        self.w0 = w[0]
        
    def predict(self, features_valid):
        return features_valid.dot(self.w) + self.w0

In [7]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550286668


Теперь проверим что результат такой же на модели с измененными данными. 

In [8]:
random_matrix = np.random.random((4, 4))       # создадим рандомную матрицу 

In [9]:
random_matrix

array([[0.70223836, 0.0577871 , 0.34943946, 0.39418858],
       [0.26937329, 0.83731566, 0.83971857, 0.81464382],
       [0.40068101, 0.8765223 , 0.13456976, 0.53256218],
       [0.10222768, 0.87803078, 0.31707162, 0.70622249]])

In [10]:
random_matrix_inv = np.linalg.inv(random_matrix)
random_matrix_inv

array([[ 0.74698583,  0.07368569,  1.56648612, -1.68322611],
       [-1.5178382 ,  1.10560867,  2.508816  , -2.3200384 ],
       [-0.800753  ,  2.49570829,  0.42847367, -2.75501636],
       [ 2.13847917, -2.50573927, -3.53827977,  5.78100384]])

In [11]:
np.allclose(np.dot(random_matrix, random_matrix_inv), np.eye(4))

True

In [12]:
np.allclose(np.dot(random_matrix_inv,random_matrix), np.eye(4))

True

Проверка показывает что матрица обратима

In [13]:
matrix = features.values                       # приведем признаки к матричному виду
features_train_new = matrix.dot(random_matrix) # получим новую матрицу путем перемножения матрицы признаков и рандомной матрицы
features_train_new.shape                       # проверим чтобы размерность матрицы осталась как в матрице признаков

(5000, 4)

In [14]:
features_new = pd.DataFrame(features_train_new, columns=features.columns)  # создадим DataFrame из новой матрицы признаков

In [15]:
model = LinearRegression()
model.fit(features_new, target)                # подставим новые признаки
predictions = model.predict(features_new)      # подставим новые признаки
print(r2_score(target, predictions))

0.42494550285976673


Мы видим что изменение матрицы Х в алгоритме обучения линейной регрессии никак не влияет на результат оценки R2. Результат оценки сохранился.

## Вывод:

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Сначала мы загрузили данные и задались вопросом: если признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? По определению работы степеней в матрицах мы пришли к выводу что в формуле линейной регрессии умножение матрицы признаков на случайную матрицу не ухудшит ее работу. 

Мы доказали что $a = a'$, где а - предсказания модели

Параметры линейной регрессии в исходной задаче и в преобразованной связаны по формуле $w' = wP^{-1}$ .

Мы составили алгоритм проверки нашей теории. Чтобы защитить персональные данные умножим матрицу с признаками клиентов на произвольную обратимую матрицу размера 4 на 4, так как количество колонок features составляет 4. В обучении и предсказании подставим вместо features новую полученную переменную. Она будет наш аналог $(XP)$
Затем сравним предсказания алгоритма с features и с новой переменной.

Затем мы сравнили полученный результат R2 метрики модели линейной регрессии для изначальной модели и измененной и получили такой же результат для R2 метрики.

Как следстиве мы получили зашифрованные данные и при этом качество модели не ухудшилось. 